In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
import numpy as np
import random
import pickle 

def scrape_goodreads_ratings(user_id, max_pages=10):
    """
    Scrape a user's star ratings from Goodreads.
    
    Args:
    - user_id (str): Goodreads user ID or profile suffix.
    - max_pages (int): Maximum number of pages to scrape (each page contains ~30 books).
    
    Returns:
    - pd.DataFrame: A DataFrame containing book titles and ratings.
    """
    base_url = f"https://www.goodreads.com/review/list/{user_id}?shelf=read"
    headers = {"User-Agent": "Mozilla/5.0"}
    books = []

    for page in range(1, max_pages + 1):
        url = f"{base_url}&page={page}"
        response = requests.get(url, headers=headers)
        
        if response.status_code != 200:
            print(f"Failed to fetch page {page}. Status code: {response.status_code}")
            break

        soup = BeautifulSoup(response.content, "html.parser")
        
        # Find all book entries in the table
        rows = soup.find_all("tr", class_="bookalike review")
        if not rows:
            print("No more data found.")
            break

        for row in rows:
            try:
                title = row.find("td", class_="field title").a.text.strip()
                rating_element = row.find("td", class_="field rating")
                rating = rating_element.find("span", class_="staticStars").get("title", "No rating")
                stars = map_rating(rating)
                books.append({"Title": title, "Rating": stars, "User_id": user_id})
#                 print(title, rating, stars)
            except AttributeError:
                # Handle rows with missing data
                continue

        print(f"Page {page} scraped successfully.")
        time.sleep(random.uniform(1, 5))  # Be kind to the server and avoid being blocked

    # Return data as a pandas DataFrame
    return pd.DataFrame(books)



In [2]:
def map_rating(phrase):
    rating_map = {
        "liked it": 3,
        "really liked it": 4,
        "it was ok": 2, 
        "it was amazing": 5, 
        "did not like it": 1,
    }
    
    return rating_map.get(phrase, "Invalid rating")  # Default to "Invalid rating" if the phrase isn't in the dictionary


In [3]:
# if __name__ == "__main__":
# #     user_id = "6688207"  # Replace with the Goodreads user ID or profile suffix
# #     for user_id in tqdm(['30181442', '75009563', '11345366', '110912303', '113964939', '11215896', '53701594', '4622890', '93628736', '176180116']):
# #     for user_id in tqdm(['2974095', '4622890', '28953843', '16174645', '4159922', '4125660', '54886546', '16912659', '260116', '4685500', '21865425']):
# #     for user_id in tqdm(['53701594', '27709782', '7566229', '16652861', '30817744', '56259255', '4125660', '60964126', 
# #                          '176167767', '28510930', '1029975', '131020767', '28862120', '88713906', '160141433', '41097916', 
# #                          '20809863', '69519261', '24017481', '7376365', '75941333', '13571407', '106618742', '17792052',
# #                          '3534528', '130656897', '7474475', '4125412', '6336365', '6026811', '3438047']):
#     for user_id in ['169695556']:
#         print("User_id = ", user_id)
#         max_pages = 30  # Adjust based on expected data
#         ratings_data = scrape_goodreads_ratings(user_id, max_pages)

#         if not ratings_data.empty:
# #             print(ratings_data.head())
# #             ratings_data.to_csv("goodreads_ratings.csv", index=False)
#             ratings_data.to_csv('goodreads_ratings.csv', mode='a', header=False, index=False)
#             print("Data saved to goodreads_ratings.csv.")
#         else:
#             print("No data retrieved.")


In [4]:
df = pd.read_csv('goodreads_ratings_series.csv')
print(df.shape)
df = df.drop_duplicates()
# Print the entire DataFrame
print(df)

duplicate_count = df['Title'].duplicated().sum()
print("Number of books with at least two people rating it:", duplicate_count)
duplicate_counts_per_value = df['Title'].value_counts()
print(duplicate_counts_per_value)
print("Number of unique books: ", df['Title'].nunique())
num_users = df['User_id'].nunique()
user_ids = list(df['User_id'].unique())
print("number of users is: ", num_users)
print("user_ids = ", user_ids)

(129645, 6)
                                                    Title Rating    User_id  \
0                                       I Am Watching You      3  169695558   
1       Three to Get Deadly\n        (Stephanie Plum, #3)      3  169695558   
2       Before the Coffee Gets Cold\n        (Before t...      4  169695558   
3       Dark Sacred Night\n        (Renée Ballard, #2;...      4  169695558   
4         Two for the Dough\n        (Stephanie Plum, #2)      4  169695558   
...                                                   ...    ...        ...   
129640                                         I, Lucifer      5    1157897   
129641                        At the Mountains of Madness      5    1157897   
129642                                        Light House      4    1157897   
129643                              Foucaultâ€™s Pendulum      4    1157897   
129644                                           Watchmen      5    1157897   

        Series  First  Suggest  
0     

In [5]:
# # Get a list of top titles in order
# top_titles = duplicate_counts_per_value.index.tolist()
# top_100 = top_titles[:100]

# for title in top_100:
#     print(title)
    
# with open("top_100.pkl", "wb") as file:
#     pickle.dump(top_100, file)

In [6]:
# threshold = 5#num_users * 0.1
# pop_titles = list(duplicate_counts_per_value[duplicate_counts_per_value > threshold].index)
# my_titles = df.loc[df["User_id"] == 169695558, "Title"].tolist()
# # print(my_titles)

# print("pop titles len = ", len(pop_titles))
# print(pop_titles)
# print("my titles len = ", len(my_titles))
# titles = list(set(pop_titles))# + my_titles))

# # #remove Harry Potter titles:
# # titles = [s for s in titles if "Harry Potter" not in s]

# num_titles = len(titles)


# print(titles)
# # print(titles)
# print("num_titles =", num_titles)

# # ratings = np.full((num_users, num_titles), None)
# ratings = np.zeros((num_users, num_titles))

# for index, row in df.iterrows():
#     if row['Title'] in titles:
#         try:
#             ratings[user_ids.index(row['User_id']), titles.index(row["Title"])] = int(row["Rating"])
# #             print("found ", row["Title"])
#         except:
#             pass
        
# print("ratings size = ", ratings.shape)
# ratings = ratings[~np.all(ratings == 0, axis=1)]
# print("ratings size = ", ratings.shape)
# # Save the list to a file
# with open("titles.pkl", "wb") as file:
#     pickle.dump(titles, file)

In [7]:
# # print(df['Suggest'])

# suggest = list(df['Suggest'])
# print(suggest)

# with open("suggest.pkl", "wb") as file:
#     suggest.dump(suggest, file)

In [8]:
# ratings_df = pd.DataFrame(ratings)
# print(ratings_df.shape)
# #delete users that don't have any of these ratings
# # ratings_df = ratings_df.loc[~(ratings_df == 0).all(axis=1)]
# ratings_df = ratings_df.loc[(ratings_df != 0).sum(axis=1) >= 4] #need at least 4 entries to stay
# print(ratings_df.shape)

# # Calculate percentage of non-zero elements
# percentage_nonzero = (np.count_nonzero(ratings_df) / ratings_df.size) * 100
# print("percentage_nonzero =", round(percentage_nonzero, 1), '%')

# # Save the list to a file
# with open("ratings_df.pkl", "wb") as file:
#     pickle.dump(ratings_df, file)


In [9]:
with open("titles.pkl", "rb") as file:
    titles = pickle.load(file)
    
with open("top_100.pkl", "rb") as file:
    top_100 = pickle.load(file)
    
with open("suggest.pkl", "rb") as file:
    suggest = pickle.load(file)
    
num_titles = len(titles)

# Load the list from the file
with open("ratings_df.pkl", "rb") as file:
    ratings_df = pickle.load(file)
    ratings = ratings_df.to_numpy()


In [10]:
# Convert the filled ratings data into a numpy array
ratings_matrix = ratings_df.values

# Initialize KNN (using user-based KNN)
import math
knn = NearestNeighbors(n_neighbors=50, metric='cosine')  # Using cosine similarity  math.ceil(num_users/10)
knn.fit(ratings_matrix)

with open("knn_model.pkl", "wb") as file:
    pickle.dump(knn, file)

# Example: Predict rating for User 0 on Item 2
user_id = 0  # Index of user in the matrix
item_id = 2  # Index of item to predict rating for

# Get the nearest neighbors for user 0 (excluding the user itself)
distances, indices = knn.kneighbors([ratings_matrix[user_id]])

pred_ratings_list = np.array([])
rankings_list = np.array([])
for item_id in range(num_titles):
    # Get the ratings for the neighbors on item 2
    neighbor_ratings = np.array([ratings_matrix[i, item_id] for i in indices[0] if not np.isnan(ratings_matrix[i, item_id])])

    predicted_rating = np.mean(neighbor_ratings[np.nonzero(neighbor_ratings)])
    rankings = np.sum(neighbor_ratings[np.nonzero(neighbor_ratings)])

#     print(predicted_rating)

    pred_ratings_list = np.append(pred_ratings_list, predicted_rating)
    rankings_list = np.append(rankings_list, rankings)
#     print(f"Predicted rating for User {user_id} on Item {item_id}: {predicted_rating}")
    
best_book_rating = np.max(pred_ratings_list)
best_book_idx = np.argmax(pred_ratings_list)

sorted_indices = np.argsort(pred_ratings_list)[::-1]
print("Top books are:")
n = 1
for i, idx in enumerate(sorted_indices): 
    if (ratings[0, idx] > 0) or (np.isnan(pred_ratings_list[idx])):
        pass
    else:
        print("#", (n) , titles[idx], "Rating:", round(pred_ratings_list[idx], 1))
        n+=1

/root/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/root/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Top books are:
# 1 Batman: Year One Rating: 5.0
# 2 It's Kind of a Funny Story Rating: 5.0
# 3 Exile
        (Forgotten Realms: The Dark Elf Trilogy, #2; Legend of Drizzt, #2) Rating: 5.0
# 4 Everything is Illuminated Rating: 5.0
# 5 The Way of Kings
        (The Stormlight Archive, #1) Rating: 5.0
# 6 Fangirl Down
        (Big Shots, #1) Rating: 5.0
# 7 Aquarium Rating: 5.0
# 8 Inferno Rating: 5.0
# 9 A Woman Is No Man Rating: 5.0
# 10 Mossflower
        (Redwall, #2) Rating: 5.0
# 11 The Neverending Story Rating: 5.0
# 12 The Love Hypothesis Rating: 5.0
# 13 The Wicked King
        (The Folk of the Air, #2) Rating: 5.0
# 14 Gemina
        (The Illuminae Files, #2) Rating: 5.0
# 15 An Ember in the Ashes
        (An Ember in the Ashes, #1) Rating: 5.0
# 16 The Ultimate Hitchhikerâ€™s Guide to the Galaxy
        (Hitchhiker's Guide to the Galaxy, #1-5) Rating: 5.0
# 17 The Black Cauldron
        (The Chronicles of Prydain, #2) Rating: 5.0
# 18 Well Met
        (Well Met, #1) Rating: 5.0

# 211 Blindness Rating: 5.0
# 212 The Passage
        (The Passage, #1) Rating: 5.0
# 213 The Shining Rating: 5.0
# 214 Cutting for Stone Rating: 5.0
# 215 Flight Behavior Rating: 5.0
# 216 If We Were Villains Rating: 5.0
# 217 Duma Key Rating: 5.0
# 218 Lover Avenged
        (Black Dagger Brotherhood, #7) Rating: 5.0
# 219 The Name of the Star
        (Shades of London, #1) Rating: 5.0
# 220 Love, Theoretically Rating: 5.0
# 221 Edge of Eternity
        (The Century Trilogy, #3) Rating: 5.0
# 222 Skin Game
        (The Dresden Files, #15) Rating: 5.0
# 223 The Institute Rating: 5.0
# 224 On Basilisk Station
        (Honor Harrington, #1) Rating: 5.0
# 225 The Night Watchman Rating: 5.0
# 226 America (The Book): A Citizen's Guide to Democracy Inaction Rating: 5.0
# 227 The Sun Is Also a Star Rating: 5.0
# 228 Can't Hurt Me: Master Your Mind and Defy the Odds Rating: 5.0
# 229 The Crystal Shard
        (Forgotten Realms: Icewind Dale, #1; Legend of Drizzt, #4) Rating: 5.0
# 230 The Wome

# 378 The Last Olympian
        (Percy Jackson and the Olympians, #5) Rating: 4.6
# 379 A Thousand Splendid Suns Rating: 4.6
# 380 The Lost Hero
        (The Heroes of Olympus, #1) Rating: 4.6
# 381 Harry Potter and the Sorcerer's Stone
        (Harry Potter, #1) Rating: 4.5
# 382 Pride and Prejudice Rating: 4.5
# 383 Rise of the Evening Star
        (Fablehaven, #2) Rating: 4.5
# 384 If You Give a Mouse a Cookie Rating: 4.5
# 385 'Salem's Lot Rating: 4.5
# 386 The True Story of the 3 Little Pigs Rating: 4.5
# 387 Fables, Vol. 1: Legends in Exile Rating: 4.5
# 388 Siege and Storm
        (The Shadow and Bone Trilogy, #2) Rating: 4.5
# 389 Winter
        (The Lunar Chronicles, #4) Rating: 4.5
# 390 He's Just Not That Into You: The No-Excuses Truth to Understanding Guys Rating: 4.5
# 391 The Secret Book of Flora Lea Rating: 4.5
# 392 The Last Child
        (Johnny Merrimon, #1) Rating: 4.5
# 393 The Dutch House Rating: 4.5
# 394 Leviathan
        (Leviathan, #1) Rating: 4.5
# 395 The Art

# 583 Along for the Ride Rating: 4.3
# 584 My Sister’s Keeper Rating: 4.3
# 585 Persepolis: The Story of a Childhood
        (Persepolis, #1) Rating: 4.3
# 586 Three Cups of Tea: One Man's Mission to Promote Peace ... One School at a Time Rating: 4.3
# 587 Faith of the Fallen
        (Sword of Truth, #6) Rating: 4.3
# 588 And Then There Were None Rating: 4.3
# 589 Wizard and Glass
        (The Dark Tower, #4) Rating: 4.3
# 590 The Brothers Karamazov Rating: 4.3
# 591 The Blade Itself
        (The First Law, #1) Rating: 4.3
# 592 Five Quarters of the Orange Rating: 4.3
# 593 Fried Green Tomatoes at the Whistle Stop Cafe Rating: 4.3
# 594 Snow Falling on Cedars Rating: 4.3
# 595 Silence
        (Hush, Hush, #3) Rating: 4.3
# 596 The Secret Hour
        (Midnighters, #1) Rating: 4.3
# 597 The Scarlet Pimpernel Rating: 4.3
# 598 Hex Hall
        (Hex Hall, #1) Rating: 4.3
# 599 I'm Glad My Mom Died Rating: 4.3
# 600 Song of Susannah
        (The Dark Tower, #6) Rating: 4.3
# 601 Flush Rati

...

        (Parque JurÃ¡sico, #1) Rating: 4.0
# 1192 Selected Poems Rating: 4.0
# 1193 From Blood and Ash
        (Blood and Ash, #1) Rating: 4.0
# 1194 The Three Musketeers Rating: 4.0
# 1195 Schindlerâ€™s List Rating: 4.0
# 1196 Test of the Twins
        (Dragonlance: Legends, #3) Rating: 4.0
# 1197 The Omnivore's Dilemma: A Natural History of Four Meals Rating: 4.0
# 1198 Goodnight Moon Rating: 4.0
# 1199 A Crown of Swords
        (The Wheel of Time, #7) Rating: 4.0
# 1200 The Clockmaker's Daughter Rating: 4.0
# 1201 This Is How It Always Is Rating: 4.0
# 1202 The Body Keeps the Score: Brain, Mind, and Body in the Healing of Trauma Rating: 4.0
# 1203 61 Hours
        (Jack Reacher, #14) Rating: 4.0
# 1204 The River of Doubt: Theodore Roosevelt's Darkest Journey Rating: 4.0
# 1205 Seabiscuit: An American Legend Rating: 4.0
# 1206 Darkfever
        (Fever, #1) Rating: 4.0
# 1207 Mexican Gothic Rating: 4.0
# 1208 The Silver Star Rating: 4.0
# 1209 One Flew Over the Cuckooâ€™s Nest Rating:

# 1435 People We Meet on Vacation Rating: 3.7
# 1436 On Writing: A Memoir of the Craft Rating: 3.7
# 1437 Dead and Gone
        (Sookie Stackhouse, #9) Rating: 3.7
# 1438 The Likeness
        (Dublin Murder Squad, #2) Rating: 3.7
# 1439 Let the Great World Spin Rating: 3.7
# 1440 The Hating Game Rating: 3.7
# 1441 One Fish, Two Fish, Red Fish, Blue Fish Rating: 3.7
# 1442 Predictably Irrational: The Hidden Forces That Shape Our Decisions Rating: 3.7
# 1443 A Bend in the Road Rating: 3.7
# 1444 The Call of the Wild Rating: 3.7
# 1445 Why Not Me? Rating: 3.7
# 1446 Shadow of Night
        (All Souls, #2) Rating: 3.7
# 1447 A Short History of Nearly Everything Rating: 3.7
# 1448 We Were Liars Rating: 3.7
# 1449 Shopaholic Takes Manhattan
        (Shopaholic, #2) Rating: 3.7
# 1450 Sword of Destiny
        (The Witcher, #0.7) Rating: 3.7
# 1451 The Reader Rating: 3.7
# 1452 The Hunchback of Notre-Dame Rating: 3.7
# 1453 The Last Battle
        (Chronicles of Narnia, #7) Rating: 3.7
# 1454 

        (The Selection, #2) Rating: 3.3
# 1659 Four to Score
        (Stephanie Plum, #4) Rating: 3.3
# 1660 The Tenth Circle Rating: 3.3
# 1661 Harriet the Spy Rating: 3.3
# 1662 Ugly Love Rating: 3.3
# 1663 The Simple Truth Rating: 3.3
# 1664 The Shipping News Rating: 3.3
# 1665 Something Blue
        (Darcy & Rachel, #2) Rating: 3.3
# 1666 Golden Son
        (Red Rising Saga, #2) Rating: 3.3
# 1667 Ninth House
        (Alex Stern, #1) Rating: 3.3
# 1668 Stiff: The Curious Lives of Human Cadavers Rating: 3.3
# 1669 Lincoln in the Bardo Rating: 3.3
# 1670 The Adventures of Huckleberry Finn Rating: 3.3
# 1671 Harry Potter and the Cursed Child: Parts One and Two
        (Harry Potter, #8) Rating: 3.3
# 1672 The Awakening Rating: 3.3
# 1673 Pretties
        (Uglies, #2) Rating: 3.3
# 1674 Of Mice and Men Rating: 3.3
# 1675 Eat, Pray, Love Rating: 3.3
# 1676 Interview with the Vampire
        (The Vampire Chronicles, #1) Rating: 3.2
# 1677 Quidditch Through the Ages Rating: 3.2
# 1678 Mr.

# 1869 A Christmas Memory Rating: 3.0
# 1870 The Rescue Rating: 3.0
# 1871 The Keeper of Lost Things Rating: 3.0
# 1872 Anna Karenina Rating: 3.0
# 1873 The Orphan Master's Son Rating: 3.0
# 1874 Camino Island
        (Camino Island, #1) Rating: 3.0
# 1875 The Silent Wife Rating: 3.0
# 1876 What Is the What Rating: 3.0
# 1877 Anne of Avonlea
        (Anne of Green Gables, #2) Rating: 3.0
# 1878 High Five
        (Stephanie Plum, #5) Rating: 3.0
# 1879 The Good Daughter
        (Good Daughter, #1) Rating: 3.0
# 1880 Cell Rating: 3.0
# 1881 Chain-Gang All-Stars Rating: 3.0
# 1882 Bloody Bones
        (Anita Blake, Vampire Hunter, #5) Rating: 3.0
# 1883 The Dressmaker's Gift Rating: 3.0
# 1884 The Paris Apartment Rating: 3.0
# 1885 Marley and Me: Life and Love With the World’s Worst Dog Rating: 3.0
# 1886 Explosive Eighteen
        (Stephanie Plum, #18) Rating: 3.0
# 1887 Like Water for Chocolate Rating: 3.0
# 1888 Home Fire Rating: 3.0
# 1889 Jemima J Rating: 3.0
# 1890 The Black Echo
  

# 2102 The New Couple in 5B Rating: 2.0
# 2103 The Woman in Cabin 10 Rating: 2.0
# 2104 Dirk Gently's Holistic Detective Agency
        (Dirk Gently, #1) Rating: 2.0
# 2105 Reckless
        (The Powerless Trilogy, #2) Rating: 2.0
# 2106 Just Mercy Rating: 2.0
# 2107 Confess Rating: 2.0
# 2108 As I Lay Dying Rating: 2.0
# 2109 The Bookish Life of Nina Hill Rating: 2.0
# 2110 Life and Other Near-Death Experiences Rating: 2.0
# 2111 One Last Stop Rating: 2.0
# 2112 Alice's Adventures in Wonderland
        (Alice's Adventures in Wonderland, #1) Rating: 2.0
# 2113 The Water Dancer Rating: 2.0
# 2114 Wide Sargasso Sea Rating: 2.0
# 2115 The Bean Trees
        (Greer Family, #1) Rating: 2.0
# 2116 Cruel Winter with You
        (Under the Mistletoe Collection, #1) Rating: 1.0
# 2117 Tangerine Rating: 1.0
# 2118 The Sum of All Fears
        (Jack Ryan, #6) Rating: 1.0
# 2119 The Black Stallion
        (The Black Stallion, #1) Rating: 1.0
# 2120 The Maidens Rating: 1.0
# 2121 Layla Rating: 1.0
#

In [25]:
#make knn for the similar user part
# Initialize KNN (using user-based KNN)
# import math
knn_30 = NearestNeighbors(n_neighbors=50, metric='cosine')  # Using cosine similarity
knn_30.fit(ratings_matrix)

with open("knn_model_30.pkl", "wb") as file:
    pickle.dump(knn_30, file)

In [12]:
rankings_list

best_book_rating = np.max(rankings_list)
best_book_idx = np.argmax(rankings_list)

sorted_indices = np.argsort(rankings_list)[::-1]
print("Top books are:")
for i, idx in enumerate(sorted_indices): 
    print("#", (i+1) , titles[idx], "Rating:", round(pred_ratings_list[idx], 1), ". Ranking:", rankings_list[idx])

Top books are:
# 1 Harry Potter and the Prisoner of Azkaban
        (Harry Potter, #3) Rating: 4.6 . Ranking: 490.0
# 2 Harry Potter and the Goblet of Fire
        (Harry Potter, #4) Rating: 4.6 . Ranking: 469.0
# 3 Harry Potter and the Deathly Hallows
        (Harry Potter, #7) Rating: 4.7 . Ranking: 462.0
# 4 Harry Potter and the Half-Blood Prince
        (Harry Potter, #6) Rating: 4.5 . Ranking: 449.0
# 5 Harry Potter and the Sorcerer's Stone
        (Harry Potter, #1) Rating: 4.5 . Ranking: 447.0
# 6 Harry Potter and the Order of the Phoenix
        (Harry Potter, #5) Rating: 4.6 . Ranking: 446.0
# 7 Harry Potter and the Chamber of Secrets
        (Harry Potter, #2) Rating: 4.4 . Ranking: 415.0
# 8 The Hunger Games
        (The Hunger Games, #1) Rating: 4.4 . Ranking: 342.0
# 9 To Kill a Mockingbird Rating: 4.0 . Ranking: 268.0
# 10 Catching Fire
        (The Hunger Games, #2) Rating: 4.4 . Ranking: 263.0
# 11 Twilight
        (The Twilight Saga, #1) Rating: 3.7 . Ranking: 253.0
# 

# 128 Speak Rating: 3.6 . Ranking: 43.0
# 129 Sense and Sensibility Rating: 3.9 . Ranking: 43.0
# 130 Brisingr
        (The Inheritance Cycle, #3) Rating: 3.9 . Ranking: 43.0
# 131 The Voyage of the Dawn Treader
        (Chronicles of Narnia, #3) Rating: 3.9 . Ranking: 43.0
# 132 The Night Circus Rating: 3.8 . Ranking: 42.0
# 133 The Son of Neptune
        (The Heroes of Olympus, #2) Rating: 4.7 . Ranking: 42.0
# 134 Artemis Fowl
        (Artemis Fowl, #1) Rating: 3.8 . Ranking: 42.0
# 135 Uglies
        (Uglies, #1) Rating: 3.7 . Ranking: 41.0
# 136 The Other Boleyn Girl
        (The Plantagenet and Tudor Novels, #9) Rating: 3.4 . Ranking: 41.0
# 137 Emma Rating: 4.1 . Ranking: 41.0
# 138 A Thousand Splendid Suns Rating: 4.6 . Ranking: 41.0
# 139 The Lost Hero
        (The Heroes of Olympus, #1) Rating: 4.6 . Ranking: 41.0
# 140 Something Borrowed
        (Darcy & Rachel, #1) Rating: 3.4 . Ranking: 41.0
# 141 Dragonfly in Amber
        (Outlander, #2) Rating: 4.0 . Ranking: 40.0
# 142

# 248 Love in the Time of Cholera Rating: 3.7 . Ranking: 26.0
# 249 Ender in Exile
        (Ender's Saga, #5) Rating: 4.3 . Ranking: 26.0
# 250 In the Woods
        (Dublin Murder Squad, #1) Rating: 3.7 . Ranking: 26.0
# 251 The Goldfinch Rating: 4.3 . Ranking: 26.0
# 252 Anne of Green Gables
        (Anne of Green Gables, #1) Rating: 4.3 . Ranking: 26.0
# 253 Interview with the Vampire
        (The Vampire Chronicles, #1) Rating: 3.2 . Ranking: 26.0
# 254 Xenocide
        (Ender's Saga, #3) Rating: 4.3 . Ranking: 26.0
# 255 The Short Second Life of Bree Tanner
        (The Twilight Saga, #3.5) Rating: 3.7 . Ranking: 26.0
# 256 Three Cups of Tea: One Man's Mission to Promote Peace ... One School at a Time Rating: 4.3 . Ranking: 26.0
# 257 Sarah's Key Rating: 3.7 . Ranking: 26.0
# 258 Where'd You Go, Bernadette Rating: 3.7 . Ranking: 26.0
# 259 Atonement Rating: 4.2 . Ranking: 25.0
# 260 This Lullaby Rating: 4.2 . Ranking: 25.0
# 261 Passion
        (Fallen, #3) Rating: 3.6 . Ranking: 2

# 394 Goodnight Moon Rating: 4.0 . Ranking: 20.0
# 395 The Lost Apothecary Rating: 4.0 . Ranking: 20.0
# 396 Under the Tuscan Sun: At Home in Italy Rating: 4.0 . Ranking: 20.0
# 397 Something Blue
        (Darcy & Rachel, #2) Rating: 3.3 . Ranking: 20.0
# 398 The Ruins of Gorlan
        (Ranger's Apprentice, #1) Rating: 5.0 . Ranking: 20.0
# 399 Homegoing Rating: 5.0 . Ranking: 20.0
# 400 Killing Lincoln: The Shocking Assassination that Changed America Forever Rating: 4.8 . Ranking: 19.0
# 401 Cloud Atlas Rating: 4.8 . Ranking: 19.0
# 402 The Gunslinger
        (The Dark Tower, #1) Rating: 3.2 . Ranking: 19.0
# 403 Crank
        (Crank, #1) Rating: 3.8 . Ranking: 19.0
# 404 The Light Between Oceans Rating: 3.2 . Ranking: 19.0
# 405 The BFG Rating: 3.8 . Ranking: 19.0
# 406 The Vanishing Half Rating: 3.8 . Ranking: 19.0
# 407 Deception Point Rating: 3.2 . Ranking: 19.0
# 408 The Invention of Hugo Cabret Rating: 3.8 . Ranking: 19.0
# 409 A Breath of Snow and Ashes
        (Outlander, #6)

# 565 Saga, Volume 2 Rating: 5.0 . Ranking: 15.0
# 566 Cold Days
        (The Dresden Files, #14) Rating: 5.0 . Ranking: 15.0
# 567 Treasure Island Rating: 3.8 . Ranking: 15.0
# 568 Mary Poppins
        (Mary Poppins, #1) Rating: 3.8 . Ranking: 15.0
# 569 Words of Radiance
        (The Stormlight Archive, #2) Rating: 5.0 . Ranking: 15.0
# 570 Dune Messiah
        (Dune, #2) Rating: 3.0 . Ranking: 15.0
# 571 The Queen's Fool
        (The Plantagenet and Tudor Novels, #12) Rating: 3.8 . Ranking: 15.0
# 572 Shanghai Girls
        (Shanghai Girls, #1) Rating: 3.8 . Ranking: 15.0
# 573 Nine Stories Rating: 5.0 . Ranking: 15.0
# 574 The Thirteenth Tale Rating: 3.8 . Ranking: 15.0
# 575 Saga, Volume 3 Rating: 5.0 . Ranking: 15.0
# 576 Anna and the French Kiss
        (Anna and the French Kiss, #1) Rating: 5.0 . Ranking: 15.0
# 577 Conversations with Friends Rating: 3.8 . Ranking: 15.0
# 578 House Rules Rating: 3.8 . Ranking: 15.0
# 579 The Guest List Rating: 3.8 . Ranking: 15.0
# 580 The Magi

...

# 2562 Three Sisters
        (The Tattooist of Auschwitz, #3) Rating: nan . Ranking: 0.0
# 2563 The Rainbow Comes and Goes: A Mother and Son on Life, Love, and Loss Rating: nan . Ranking: 0.0
# 2564 The Secret
        (The Secret, #1) Rating: nan . Ranking: 0.0
# 2565 The Care and Feeding of Ravenously Hungry Girls Rating: nan . Ranking: 0.0
# 2566 Mr. Maybe Rating: nan . Ranking: 0.0
# 2567 We Are Not Like Them Rating: nan . Ranking: 0.0
# 2568 The Essex Serpent Rating: nan . Ranking: 0.0
# 2569 In a Dark, Dark Wood Rating: nan . Ranking: 0.0
# 2570 Three Wishes Rating: nan . Ranking: 0.0
# 2571 The Woman in Black Rating: nan . Ranking: 0.0
# 2572 Old Man's War
        (Old Man's War, #1) Rating: nan . Ranking: 0.0
# 2573 The Witch Elm Rating: nan . Ranking: 0.0
# 2574 What Remains Rating: nan . Ranking: 0.0
# 2575 Lie to Me Rating: nan . Ranking: 0.0
# 2576 Zeitoun Rating: nan . Ranking: 0.0
# 2577 Miracle Creek Rating: nan . Ranking: 0.0
# 2578 The Love of My Afterlife Rating: nan .

# 2707 Behold the Dreamers Rating: nan . Ranking: 0.0
# 2708 American Pastoral Rating: nan . Ranking: 0.0
# 2709 The Passenger Rating: nan . Ranking: 0.0
# 2710 I Let You Go Rating: nan . Ranking: 0.0
# 2711 Wintersmith
        (Discworld, #35; Tiffany Aching, #3) Rating: nan . Ranking: 0.0
# 2712 To Green Angel Tower
        (Memory, Sorrow, and Thorn, #3) Rating: nan . Ranking: 0.0
# 2713 The Mystery of Edwin Drood Rating: nan . Ranking: 0.0
# 2714 The Deep Rating: nan . Ranking: 0.0
# 2715 Five Little Indians Rating: nan . Ranking: 0.0
# 2716 The Year of the Flood
        (MaddAddam, #2) Rating: nan . Ranking: 0.0
# 2717 Fractured
        (Will Trent, #2) Rating: nan . Ranking: 0.0
# 2718 Ready or Not Rating: nan . Ranking: 0.0
# 2719 Monogamy Rating: nan . Ranking: 0.0
# 2720 The Marriage Portrait Rating: nan . Ranking: 0.0
# 2721 Babel Rating: nan . Ranking: 0.0
# 2722 We the Living Rating: nan . Ranking: 0.0
# 2723 Romantic Comedy Rating: nan . Ranking: 0.0
# 2724 Absolution Rati

# 2882 Once There Were Wolves Rating: nan . Ranking: 0.0
# 2883 By Any Other Name Rating: nan . Ranking: 0.0
# 2884 Small Things Like These Rating: nan . Ranking: 0.0
# 2885 Home Before Dark Rating: nan . Ranking: 0.0
# 2886 A Single Man Rating: nan . Ranking: 0.0
# 2887 The Housemaid's Secret
        (The Housemaid, #2) Rating: nan . Ranking: 0.0
# 2888 Odd and the Frost Giants Rating: nan . Ranking: 0.0
# 2889 Tender Is the Flesh Rating: nan . Ranking: 0.0
# 2890 Before I Let Go
        (Skyland, #1) Rating: nan . Ranking: 0.0
# 2891 First Lord's Fury
        (Codex Alera, #6) Rating: nan . Ranking: 0.0
# 2892 Small Gods
        (Discworld, #13) Rating: nan . Ranking: 0.0
# 2893 The Block Party Rating: nan . Ranking: 0.0
# 2894 Rendezvous with Rama
        (Rama, #1) Rating: nan . Ranking: 0.0
# 2895 Redhead by the Side of the Road Rating: nan . Ranking: 0.0
# 2896 A Fire Upon the Deep
        (Zones of Thought, #1) Rating: nan . Ranking: 0.0
# 2897 Next of Kin Rating: nan . Ranking:

# 3059 Seven Days in June Rating: nan . Ranking: 0.0
# 3060 Endurance: Shackleton's Incredible Voyage Rating: nan . Ranking: 0.0
# 3061 The Night Ship Rating: nan . Ranking: 0.0
# 3062 Post Office Rating: nan . Ranking: 0.0
# 3063 Canada Rating: nan . Ranking: 0.0
# 3064 Lone Wolf Rating: nan . Ranking: 0.0
# 3065 This Is How You Lose Her Rating: nan . Ranking: 0.0
# 3066 4:50 from Paddington
        (Miss Marple, #7) Rating: nan . Ranking: 0.0
# 3067 The Kind Worth Saving
        (Henry Kimball/Lily Kintner, #2) Rating: nan . Ranking: 0.0
# 3068 Happiness Falls Rating: nan . Ranking: 0.0
# 3069 The Patron Saint of Liars Rating: nan . Ranking: 0.0
# 3070 Eight Perfect Murders Rating: nan . Ranking: 0.0
# 3071 Our Missing Hearts Rating: nan . Ranking: 0.0
# 3072 The Ashes & the Star-Cursed King
        (Crowns of Nyaxia, #2) Rating: nan . Ranking: 0.0
# 3073 The Wife Rating: nan . Ranking: 0.0
# 3074 Unsheltered Rating: nan . Ranking: 0.0
# 3075 Half the Sky: Turning Oppression into Opp

# 3230 Lula Dean's Little Library of Banned Books Rating: nan . Ranking: 0.0
# 3231 The Perfect Storm: A True Story of Men Against the Sea Rating: nan . Ranking: 0.0
# 3232 The Mystery Guest
        (Molly the Maid, #2) Rating: nan . Ranking: 0.0
# 3233 A Doll's House Rating: nan . Ranking: 0.0
# 3234 Unsettled Ground Rating: nan . Ranking: 0.0
# 3235 The Plot Against America Rating: nan . Ranking: 0.0
# 3236 Monster Rating: nan . Ranking: 0.0
# 3237 The Moonstone Rating: nan . Ranking: 0.0
# 3238 These Precious Days: Essays Rating: nan . Ranking: 0.0
# 3239 Soul Harvest: The World Takes Sides
        (Left Behind, #4) Rating: nan . Ranking: 0.0


In [13]:
#give a list sorted out with books you've already read:
# sorted_indices = np.argsort(pred_ratings_list)[::-1]
print("Top books are:")
list_num = 1
for idx in sorted_indices: 
#     print("ratings_matrix[user_id, idx]= ", ratings_matrix[user_id, idx])
    if (ratings_matrix[user_id, idx] > 0) or (np.isnan(pred_ratings_list[idx])):
        continue
    print("#", list_num , titles[idx], "Rating:", round(pred_ratings_list[idx], 1))
    list_num += 1

Top books are:
# 1 Harry Potter and the Sorcerer's Stone
        (Harry Potter, #1) Rating: 4.5
# 2 The Great Gatsby Rating: 3.7
# 3 Eclipse
        (The Twilight Saga, #3) Rating: 3.7
# 4 Breaking Dawn
        (The Twilight Saga, #4) Rating: 3.6
# 5 The da Vinci Code
        (Robert Langdon, #2) Rating: 3.8
# 6 Pride and Prejudice Rating: 4.5
# 7 The Hobbit, or There and Back Again
        (The Lord of the Rings, #0) Rating: 4.2
# 8 Lord of the Flies Rating: 3.5
# 9 Little Women Rating: 4.0
# 10 A Game of Thrones
        (A Song of Ice and Fire, #1) Rating: 4.8
# 11 The Fellowship of the Ring
        (The Lord of the Rings, #1) Rating: 4.1
# 12 The Lovely Bones Rating: 3.7
# 13 Of Mice and Men Rating: 3.3
# 14 The Lion, the Witch and the Wardrobe
        (Chronicles of Narnia, #1) Rating: 4.2
# 15 1984 Rating: 4.1
# 16 The Girl with the Dragon Tattoo
        (Millennium, #1) Rating: 4.3
# 17 Water for Elephants Rating: 4.4
# 18 The Kite Runner Rating: 4.0
# 19 Fahrenheit 451 Rating: 4

        (Throne of Glass, #1) Rating: 4.1
# 173 A Man Called Ove Rating: 4.1
# 174 The Magicianâ€™s Nephew
        (Chronicles of Narnia, #6) Rating: 3.6
# 175 Hatchet
        (Brian's Saga, #1) Rating: 3.6
# 176 Iron Flame
        (The Empyrean, #2) Rating: 4.1
# 177 Legend
        (Legend, #1) Rating: 4.8
# 178 Educated Rating: 4.1
# 179 Fifty Shades Freed
        (Fifty Shades, #3) Rating: 2.9
# 180 Fifty Shades Darker
        (Fifty Shades, #2) Rating: 3.2
# 181 The Secret Garden Rating: 4.0
# 182 Harry Potter Series Box Set
        (Harry Potter, #1-7) Rating: 4.7
# 183 The Old Man and the Sea Rating: 3.1
# 184 Tempted
        (House of Night, #6) Rating: 4.0
# 185 Inheritance
        (The Inheritance Cycle, #4) Rating: 4.7
# 186 The Tales of Beedle the Bard
        (Hogwarts Library, #3) Rating: 4.0
# 187 Circe Rating: 4.7
# 188 The Girl Who Kicked the Hornetâ€™s Nest
        (Millennium, #3) Rating: 3.5
# 189 American Gods
        (American Gods, #1) Rating: 4.7
# 190 The Joy Lu

# 348 Cloud Atlas Rating: 4.8
# 349 The Gunslinger
        (The Dark Tower, #1) Rating: 3.2
# 350 Crank
        (Crank, #1) Rating: 3.8
# 351 The Light Between Oceans Rating: 3.2
# 352 The BFG Rating: 3.8
# 353 The Vanishing Half Rating: 3.8
# 354 Deception Point Rating: 3.2
# 355 The Invention of Hugo Cabret Rating: 3.8
# 356 Running with Scissors Rating: 2.7
# 357 Station Eleven Rating: 3.8
# 358 When Breath Becomes Air Rating: 4.8
# 359 A Walk in the Woods: Rediscovering America on the Appalachian Trail Rating: 3.8
# 360 The Tell-Tale Heart and Other Writings Rating: 3.8
# 361 The Mark of Athena
        (The Heroes of Olympus, #3) Rating: 4.8
# 362 Sunshine Rating: 4.8
# 363 A Child Called "It"
        (Dave Pelzer, #1) Rating: 3.8
# 364 Fool Moon
        (The Dresden Files, #2) Rating: 4.8
# 365 Sheâ€™s Come Undone Rating: 3.8
# 366 March Rating: 3.8
# 367 Ender's Shadow
        (The Shadow Series, #1) Rating: 4.8
# 368 Catch-22 Rating: 2.7
# 369 Mistborn: The Final Empire
        

# 507 Saga, Volume 2 Rating: 5.0
# 508 Cold Days
        (The Dresden Files, #14) Rating: 5.0
# 509 Treasure Island Rating: 3.8
# 510 Mary Poppins
        (Mary Poppins, #1) Rating: 3.8
# 511 Words of Radiance
        (The Stormlight Archive, #2) Rating: 5.0
# 512 Dune Messiah
        (Dune, #2) Rating: 3.0
# 513 The Queen's Fool
        (The Plantagenet and Tudor Novels, #12) Rating: 3.8
# 514 Shanghai Girls
        (Shanghai Girls, #1) Rating: 3.8
# 515 Nine Stories Rating: 5.0
# 516 The Thirteenth Tale Rating: 3.8
# 517 Saga, Volume 3 Rating: 5.0
# 518 Anna and the French Kiss
        (Anna and the French Kiss, #1) Rating: 5.0
# 519 Conversations with Friends Rating: 3.8
# 520 House Rules Rating: 3.8
# 521 The Guest List Rating: 3.8
# 522 The Magician’s Nephew
        (Chronicles of Narnia, #6) Rating: 3.8
# 523 Go Set a Watchman Rating: 3.8
# 524 The Pearl Rating: 3.0
# 525 The Amulet of Samarkand
        (Bartimaeus, #1) Rating: 3.8
# 526 Sabriel
        (Abhorsen,  #1) Rating: 5.

# 666 The House of Mirth Rating: 4.3
# 667 Wolves of the Calla
        (The Dark Tower, #5) Rating: 4.3
# 668 Silence
        (Hush, Hush, #3) Rating: 4.3
# 669 The Overstory Rating: 4.3
# 670 Along for the Ride Rating: 4.3
# 671 Ender's Game
        (Ender's Saga, #1) Rating: 4.3
# 672 The Brothers Karamazov Rating: 4.3
# 673 Gilead
        (Gilead, #1) Rating: 4.3
# 674 Into Thin Air: A Personal Account of the Mt. Everest Disaster Rating: 4.3
# 675 The Opal Deception
        (Artemis Fowl, #4) Rating: 4.3
# 676 The Amazing Adventures of Kavalier & Clay Rating: 4.0
# 677 The Tao of Pooh Rating: 4.0
# 678 Destroy Me
        (Shatter Me, #1.5) Rating: 4.0
# 679 Pandemonium
        (Delirium, #2) Rating: 4.0
# 680 Before We Were Yours Rating: 4.0
# 681 Under the Banner of Heaven: A Story of Violent Faith Rating: 4.0
# 682 Wicked Lovely
        (Wicked Lovely, #1) Rating: 3.0
# 683 The Casual Vacancy Rating: 4.0
# 684 Such a Fun Age Rating: 4.0
# 685 The Last Song Rating: 4.0
# 686 Leaves

...

        (Anita Blake, Vampire Hunter, #9) Rating: 2.5
# 1377 Under the Whispering Door Rating: 5.0
# 1378 Lies My Teacher Told Me: Everything Your American History Textbook Got Wrong Rating: 5.0
# 1379 Plainsong
        (Plainsong, #1) Rating: 5.0
# 1380 The Story of Ferdinand Rating: 5.0
# 1381 As You Like It Rating: 5.0
# 1382 The Institute Rating: 5.0
# 1383 The Fine Print
        (Dreamland Billionaires, #1) Rating: 5.0
# 1384 I Know This Much Is True Rating: 5.0
# 1385 Nora Goes Off Script Rating: 5.0
# 1386 The Chain Rating: 5.0
# 1387 All My Puny Sorrows Rating: 5.0
# 1388 Barrel Fever: Stories and Essays Rating: 2.5
# 1389 The Girl You Left Behind Rating: 5.0
# 1390 Death's End
        (Remembrance of Earthâ€™s Past, #3) Rating: 5.0
# 1391 On Tyranny: Twenty Lessons from the Twentieth Century Rating: 5.0
# 1392 Running Blind
        (Jack Reacher, #4) Rating: 5.0
# 1393 The Queen of the Damned
        (The Vampire Chronicles, #3) Rating: 5.0
# 1394 The Rose Code Rating: 5.0
# 1

        (The Walking Dead, #1) Rating: 5.0
# 1540 In Defense of Food: An Eater's Manifesto Rating: 5.0
# 1541 Braiding Sweetgrass: Indigenous Wisdom, Scientific Knowledge, and the Teachings of Plants Rating: 5.0
# 1542 Slow Dance Rating: 5.0
# 1543 Inside Out Rating: 4.0
# 1544 The Book of Longings Rating: 4.0
# 1545 Fortunately, the Milk Rating: 4.0
# 1546 Beautiful Darkness
        (Caster Chronicles, #2) Rating: 4.0
# 1547 The Art Forger Rating: 4.0
# 1548 The Last Mrs. Parrish
        (Mrs. Parrish, #1) Rating: 4.0
# 1549 Reminders of Him Rating: 4.0
# 1550 The Mists of Avalon
        (Avalon, #1) Rating: 4.0
# 1551 Blink: The Power of Thinking Without Thinking Rating: 4.0
# 1552 Leaving Time Rating: 4.0
# 1553 The Second Summer of the Sisterhood
        (Sisterhood, #2) Rating: 4.0
# 1554 Magician's Gambit
        (The Belgariad #3) Rating: 4.0
# 1555 The Phantom of the Opera Rating: 4.0
# 1556 The Man Who Mistook His Wife for a Hat and Other Clinical Tales Rating: 4.0
# 1557 Hear

# 1693 Saturday Rating: 4.0
# 1694 The Path of Daggers
        (The Wheel of Time, #8) Rating: 4.0
# 1695 61 Hours
        (Jack Reacher, #14) Rating: 4.0
# 1696 Wandering Stars Rating: 4.0
# 1697 Cloud Cuckoo Land Rating: 4.0
# 1698 Now You See Her Rating: 4.0
# 1699 The Peach Keeper Rating: 4.0
# 1700 The Sanatorium
        (Detective Elin Warner, #1) Rating: 4.0
# 1701 Quiet: The Power of Introverts in a World That Can't Stop Talking Rating: 4.0
# 1702 The Only One Left Rating: 4.0
# 1703 The Stationery Shop Rating: 4.0
# 1704 The Miraculous Journey of Edward Tulane Rating: 4.0
# 1705 The Brethren Rating: 4.0
# 1706 Stillhouse Lake
        (Stillhouse Lake, #1) Rating: 4.0
# 1707 The Cider House Rules Rating: 4.0
# 1708 West With Giraffes Rating: 4.0
# 1709 Nine Perfect Strangers Rating: 4.0
# 1710 This is Where I Leave You Rating: 4.0
# 1711 The God of Small Things Rating: 4.0
# 1712 Long Way Down Rating: 4.0
# 1713 Brideshead Revisited Rating: 4.0
# 1714 Run for Your Life
        

# 1870 Madame Bovary Rating: 3.0
# 1871 Dubliners Rating: 3.0
# 1872 Cell Rating: 3.0
# 1873 Valley of the Dolls Rating: 3.0
# 1874 The Good Daughter
        (Good Daughter, #1) Rating: 3.0
# 1875 The Sociopath Next Door Rating: 3.0
# 1876 Still Life with Woodpecker Rating: 3.0
# 1877 The Orphan's Tale Rating: 3.0
# 1878 The Running Man Rating: 3.0
# 1879 The Glass Hotel Rating: 3.0
# 1880 Suzanne's Diary for Nicholas Rating: 3.0
# 1881 It Happened One Summer
        (Bellinger Sisters, #1) Rating: 3.0
# 1882 The Only Survivors Rating: 3.0
# 1883 The Last Juror Rating: 3.0
# 1884 The Killing Dance
        (Anita Blake, Vampire Hunter, #6) Rating: 3.0
# 1885 The Snow Child Rating: 3.0
# 1886 A Lovely Lie Rating: 3.0
# 1887 Fever Rating: 3.0
# 1888 The Quickie Rating: 3.0
# 1889 Like Water for Chocolate Rating: 3.0
# 1890 Chronicle of a Death Foretold Rating: 3.0
# 1891 Dragonflight
        (Dragonriders of Pern, #1) Rating: 3.0
# 1892 At the Water's Edge Rating: 3.0
# 1893 The Club Rati

        (Campus Diaries, #1) Rating: 3.0
# 2049 The Doors of Perception & Heaven and Hell Rating: 3.0
# 2050 Fear and Loathing in Las Vegas Rating: 3.0
# 2051 The Strange Case of Dr. Jekyll and Mr. Hyde and Other Tales of Terror Rating: 3.0
# 2052 The 7 1/2 Deaths of Evelyn Hardcastle Rating: 3.0
# 2053 The Outsider Rating: 3.0
# 2054 Mudbound Rating: 3.0
# 2055 Luckiest Girl Alive Rating: 3.0
# 2056 North Woods Rating: 3.0
# 2057 The Brass Verdict
        (The Lincoln Lawyer, #2; Harry Bosch Universe, #19) Rating: 3.0
# 2058 The Rainmaker Rating: 3.0
# 2059 The 7 Habits of Highly Effective People: Powerful Lessons in Personal Change Rating: 3.0
# 2060 Debt of Honor
        (Jack Ryan, #7) Rating: 3.0
# 2061 Roses Are Red
        (Alex Cross, #6) Rating: 3.0
# 2062 Geek Love Rating: 3.0
# 2063 The Leftovers Rating: 3.0
# 2064 Interpreter of Maladies Rating: 3.0
# 2065 Brida Rating: 3.0
# 2066 The Secret History Rating: 3.0
# 2067 90 Minutes in Heaven: A True Story of Death and Life Rat

In [14]:
idx = 8
this_ratings = ratings_matrix[indices[0,idx]]
print(this_ratings)

my_ratings = ratings_matrix[indices[0,0]]
print(my_ratings)

for i, rating in enumerate(this_ratings):
    if rating > 0 and my_ratings[i]>0:
        print("-", titles[i], ", their Rating:", rating, " My Rating:", my_ratings[i])

# print(distances, indices[-1,-1])

[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
- Twilight
        (The Twilight Saga, #1) , their Rating: 3.0  My Rating: 3.0
- Harry Potter and the Goblet of Fire
        (Harry Potter, #4) , their Rating: 5.0  My Rating: 5.0
- Harry Potter and the Chamber of Secrets
        (Harry Potter, #2) , their Rating: 4.0  My Rating: 5.0
- The Giver
        (The Giver, #1) , their Rating: 3.0  My Rating: 4.0
- Harry Potter and the Half-Blood Prince
        (Harry Potter, #6) , their Rating: 3.0  My Rating: 5.0
- Harry Potter and the Prisoner of Azkaban
        (Harry Potter, #3) , their Rating: 4.0  My Rating: 5.0
- Harry Potter and the Order of the Phoenix
        (Harry Potter, #5) , their Rating: 5.0  My Rating: 5.0
- The Hunger Games
        (The Hunger Games, #1) , their Rating: 4.0  My Rating: 5.0
- Harry Potter and the Deathly Hallows
        (Harry Potter, #7) , their Rating: 5.0  My Rating: 5.0
- The Lightning Thief
        (Percy Jackson and the Olympians, #1) , their Rating: 4.0  M

In [15]:
#find most similar books using cosine similarity
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(ratings_matrix.T)

similarity_df = pd.DataFrame(similarity_matrix, index=titles, columns=titles)

# Function to get k nearest neighbors for a movie
def get_similar_book(book_name, k=3):
    similar_book = similarity_df[book_name].sort_values(ascending=False)[1:k+1]
    return similar_book

book_name = 'First Lie Wins'
print("\nTop 5 similar book to", book_name, ":")
print(get_similar_book(book_name, k=5))


Top 5 similar book to First Lie Wins :
She's Not Sorry                0.679136
Listen for the Lie             0.665685
Darling Girls                  0.658347
The Fury                       0.642380
The Last One at the Wedding    0.608053
Name: First Lie Wins, dtype: float64


In [16]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer

# Example user rating data (rows = users, columns = items)
ratings_df = pd.DataFrame(ratings)

# Step 1: Handle missing values using imputation (we will fill missing values with the mean rating)
imputer = SimpleImputer(strategy='mean')
ratings_filled = imputer.fit_transform(ratings_df)

# Step 2: Apply KMeans clustering
# We will use 2 clusters as an example
kmeans = KMeans(n_clusters=int(num_users/10), random_state=42)
clusters = kmeans.fit_predict(ratings_filled)

# Step 3: Add the cluster labels to the original DataFrame
ratings_df['Cluster'] = clusters

# # Print the user ratings with cluster assignments
# print("\nUser Ratings with Clusters:")
# print(ratings_df)

# # Step 4: Print the cluster centers (the centroid of each cluster)
# print("\nCluster Centers (Centroids):")
# print(kmeans.cluster_centers_)


user_id = 0
# print(clusters)

cluster_this_user = clusters[user_id]
# print(cluster_this_user)

pred_ratings_list = kmeans.cluster_centers_[user_id]
# print(pred_ratings_list)
# for i in range(len())

#give a list sorted out with books you've already read:
sorted_indices = np.argsort(pred_ratings_list)[::-1]
print("Top books are:")
list_num = 1
for idx in sorted_indices: 
#     print("ratings_matrix[user_id, idx]= ", ratings_matrix[user_id, idx])
    if (ratings_matrix[user_id, idx] > 0) or (np.isnan(pred_ratings_list[idx])) :
        continue
    print("#", list_num , titles[idx], "Rating:", round(pred_ratings_list[idx], 1))
    list_num += 1

/root/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Top books are:
# 1 Eclipse
        (The Twilight Saga, #3) Rating: 4.3
# 2 Breaking Dawn
        (The Twilight Saga, #4) Rating: 3.9
# 3 Harry Potter and the Sorcerer's Stone
        (Harry Potter, #1) Rating: 1.4
# 4 The Notebook
        (The Notebook, #1) Rating: 0.9
# 5 The da Vinci Code
        (Robert Langdon, #2) Rating: 0.8
# 6 The Alchemist Rating: 0.7
# 7 The Diary of a Young Girl Rating: 0.6
# 8 Confessions of a Shopaholic
        (Shopaholic, #1) Rating: 0.6
# 9 Little Women Rating: 0.6
# 10 City of Bones
        (The Mortal Instruments, #1) Rating: 0.6
# 11 Betrayed
        (House of Night, #2) Rating: 0.5
# 12 Marked
        (House of Night, #1) Rating: 0.5
# 13 Untamed
        (House of Night, #4) Rating: 0.5
# 14 The Devil Wears Prada
        (The Devil Wears Prada, #1) Rating: 0.5
# 15 Memoirs of a Geisha Rating: 0.5
# 16 The Short Second Life of Bree Tanner
        (The Twilight Saga, #3.5) Rating: 0.5
# 17 The Hobbit, or There and Back Again
        (The Lord of the R

# 208 Bared to You
        (Crossfire, #1) Rating: 0.1
# 209 Ghosts Rating: 0.1
# 210 Green Eggs and Ham Rating: 0.1
# 211 Entwined with You
        (Crossfire, #3) Rating: 0.1
# 212 Sunshine Rating: 0.1
# 213 Love You Forever Rating: 0.1
# 214 Columbine Rating: 0.1
# 215 The Women Rating: 0.1
# 216 Purple Hibiscus Rating: 0.1
# 217 I Am Number Four
        (Lorien Legacies, #1) Rating: 0.1
# 218 Crescendo
        (Hush, Hush, #2) Rating: 0.1
# 219 Rise of the Evening Star
        (Fablehaven, #2) Rating: 0.1
# 220 The Duke and I
        (Bridgertons, #1) Rating: 0.1
# 221 Lover Revealed
        (Black Dagger Brotherhood, #4) Rating: 0.1
# 222 Lover Avenged
        (Black Dagger Brotherhood, #7) Rating: 0.1
# 223 The Selection
        (The Selection, #1) Rating: 0.1
# 224 A Woman Is No Man Rating: 0.1
# 225 Frankenstein Rating: 0.1
# 226 My Brilliant Friend
        (My Brilliant Friend, #1) Rating: 0.1
# 227 Into the Wilderness
        (Wilderness, #1) Rating: 0.1
# 228 Fables, Vol. 9:

        (Sisterhood, #4) Rating: 0.1
# 373 Interview with the Vampire
        (The Vampire Chronicles, #1) Rating: 0.1
# 374 Black and Blue Rating: 0.1
# 375 The Angel Experiment
        (Maximum Ride, #1) Rating: 0.1
# 376 Freakonomics: A Rogue Economist Explores the Hidden Side of Everything Rating: 0.1
# 377 Cell Rating: 0.1
# 378 The Elfstones of Shannara
        (The Original Shannara Trilogy, #2) Rating: 0.1
# 379 Dress Your Family in Corduroy and Denim Rating: 0.1
# 380 Becoming Rating: 0.1
# 381 A Caress of Twilight
        (Merry Gentry, #2) Rating: 0.1
# 382 The Perks of Being a Wallflower Rating: 0.1
# 383 Ringworld
        (Ringworld, #1) Rating: 0.1
# 384 Eleanor & Park Rating: 0.1
# 385 She Said: Breaking the Sexual Harassment Story That Helped Ignite a Movement Rating: 0.1
# 386 Inkheart
        (Inkworld, #1) Rating: 0.1
# 387 Change of Heart Rating: 0.1
# 388 Dragonflight
        (Dragonriders of Pern, #1) Rating: 0.1
# 389 A Fistful of Charms
        (The Hollows, #4)

# 613 The Wives Rating: 0.0
# 614 The Amazing Adventures of Kavalier & Clay Rating: 0.0
# 615 We Are the Brennans Rating: 0.0
# 616 The Great Divorce Rating: 0.0
# 617 Part of Your World
        (Part of Your World, #1) Rating: 0.0
# 618 The BFG Rating: 0.0
# 619 Then She Was Gone Rating: 0.0
# 620 The Accident Rating: 0.0
# 621 Rock Paper Scissors Rating: 0.0
# 622 Leviathan Wakes
        (The Expanse, #1) Rating: 0.0
# 623 Mexican Gothic Rating: 0.0
# 624 Being Mortal: Medicine and What Matters in the End Rating: 0.0
# 625 I, Robot
        (Robot, #0.1) Rating: 0.0
# 626 The Return of Ellie Black Rating: 0.0
# 627 The Nickel Boys Rating: 0.0
# 628 Home Before Dark Rating: 0.0
# 629 Hyperion
        (Hyperion Cantos, #1) Rating: 0.0
# 630 Eye of the Needle Rating: 0.0
# 631 The Cider House Rules Rating: 0.0
# 632 Once There Were Wolves Rating: 0.0
# 633 The History of Love Rating: 0.0
# 634 The Assassin's Blade
        (Throne of Glass, #0.1-0.5) Rating: 0.0
# 635 Cloud Cuckoo Land Ra

...

# 2403 Changeless
        (Parasol Protectorate, #2) Rating: -0.0
# 2404 Deadlocked
        (Sookie Stackhouse, #12) Rating: -0.0
# 2405 Come as You Are: The Surprising New Science that Will Transform Your Sex Life Rating: -0.0
# 2406 Don't Let the Pigeon Drive the Bus! Rating: -0.0
# 2407 Danse Macabre Rating: -0.0
# 2408 The One Minute Manager Rating: -0.0
# 2409 Persuader
        (Jack Reacher, #7) Rating: -0.0
# 2410 Quidditch Through the Ages Rating: -0.0
# 2411 Behold the Dreamers Rating: -0.0
# 2412 We Were Never Here Rating: -0.0
# 2413 Pandora
        (New Tales of the Vampires, #1) Rating: -0.0
# 2414 Luckiest Girl Alive Rating: -0.0
# 2415 Sword of Destiny
        (The Witcher, #0.7) Rating: -0.0
# 2416 Erotic Stories for Punjabi Widows Rating: -0.0
# 2417 Lost in a Good Book
        (Thursday Next, #2) Rating: -0.0
# 2418 Pieces of Her
        (Andrea Oliver, #1) Rating: -0.0
# 2419 The Long Weekend Rating: -0.0
# 2420 Camino Island
        (Camino Island, #1) Rating: -0.0


        (Brian's Saga, #1) Rating: -0.0
# 2593 The Witching Hour
        (Lives of the Mayfair Witches, #1) Rating: -0.0
# 2594 The Lion Women of Tehran Rating: -0.0
# 2595 Zero Days Rating: -0.0
# 2596 Nine Perfect Strangers Rating: -0.0
# 2597 Dark Lover
        (Black Dagger Brotherhood, #1) Rating: -0.0
# 2598 Darling Rose Gold Rating: -0.0
# 2599 A Kingdom of Flesh and Fire
        (Blood and Ash, #2) Rating: -0.0
# 2600 The Book Woman's Daughter
        (The Book Woman of Troublesome Creek, #2) Rating: -0.0
# 2601 The Professor and the Madman: A Tale of Murder, Insanity and the Making of the Oxford English Dictionary Rating: -0.0
# 2602 I Am America Rating: -0.0
# 2603 The Library Book Rating: -0.0
# 2604 The Simple Truth Rating: -0.0
# 2605 Garden Spells
        (Waverley Family, #1) Rating: -0.0
# 2606 Towers of Midnight
        (The Wheel of Time, #13) Rating: -0.0
# 2607 If You Give a Moose a Muffin Rating: -0.0
# 2608 Johnny Got His Gun Rating: -0.0
# 2609 The Night Shift Ra

# 2753 The Little Paris Bookshop Rating: -0.0
# 2754 Dreams of Joy
        (Shanghai Girls, #2) Rating: -0.0
# 2755 No One Can Know Rating: -0.0
# 2756 Duma Key Rating: -0.0
# 2757 The Last One Rating: -0.0
# 2758 The Gunslinger Born
        (Stephen King's The Dark Tower: Beginnings, #1) Rating: -0.0
# 2759 The White Queen
        (The Plantagenet and Tudor Novels, #2) Rating: -0.0
# 2760 The Love Songs of W.E.B. Du Bois Rating: -0.0
# 2761 The Summons Rating: -0.0
# 2762 Still Life with Woodpecker Rating: -0.0
# 2763 The First Ladies Rating: -0.0
# 2764 Dolores Claiborne Rating: -0.0
# 2765 Rage Rating: -0.0
# 2766 My Evil Mother Rating: -0.0
# 2767 A Knight of the Seven Kingdoms
        (The Tales of Dunk and Egg, #1-3) Rating: -0.0
# 2768 Selected Poems Rating: -0.0
# 2769 Library of Souls
        (Miss Peregrine's Peculiar Children, #3) Rating: -0.0
# 2770 Well Met
        (Well Met, #1) Rating: -0.0
# 2771 Notes from Underground, White Nights, The Dream of a Ridiculous Man, and S

# 2944 The Dilemma Rating: -0.0
# 2945 Three Junes Rating: -0.0
# 2946 Walden Rating: -0.0
# 2947 Blindness Rating: -0.0
# 2948 Pet Sematary Rating: -0.0
# 2949 A Little Princess Rating: -0.0
# 2950 Keep It in the Family Rating: -0.0
# 2951 Queen of Shadows
        (Throne of Glass, #4) Rating: -0.0
# 2952 Furies of Calderon
        (Codex Alera, #1) Rating: -0.0
# 2953 Morality for Beautiful Girls
        (No. 1 Ladies' Detective Agency, #3) Rating: -0.0
# 2954 The House of Hades
        (The Heroes of Olympus, #4) Rating: -0.0
# 2955 Timeline Rating: -0.0
# 2956 Foundation
        (Foundation, #1) Rating: -0.0
# 2957 Calypso Rating: -0.0
# 2958 The Family Game Rating: -0.0
# 2959 Children of Dune
        (Dune #3) Rating: -0.0
# 2960 Two Nights in Lisbon Rating: -0.0
# 2961 The Overstory Rating: -0.0
# 2962 Landline Rating: -0.0
# 2963 Mr. Penumbra's 24-Hour Bookstore
        (Mr. Penumbra's 24-Hour Bookstore, #1) Rating: -0.0
# 2964 The Hike Rating: -0.0
# 2965 The God of Small Thin

# 3140 A Light in the Attic Rating: -0.0
# 3141 Hamnet Rating: -0.0
# 3142 The Color of Magic
        (Discworld, #1; Rincewind, #1) Rating: -0.0
# 3143 The Paris Apartment Rating: -0.0
# 3144 Where the Crawdads Sing Rating: -0.0
# 3145 Watchmen Rating: -0.0
# 3146 Funny Story Rating: -0.0
# 3147 The Lost Apothecary Rating: -0.0
# 3148 It Starts with Us
        (It Ends with Us, #2) Rating: -0.0
# 3149 Remarkably Bright Creatures Rating: -0.0
# 3150 The Four Winds Rating: -0.0
# 3151 I Know This Much Is True Rating: -0.0
# 3152 When Breath Becomes Air Rating: -0.0
# 3153 Things Fall Apart
        (The African Trilogy, #1) Rating: -0.0
# 3154 One Hundred Years of Solitude Rating: -0.0
# 3155 Lessons in Chemistry Rating: -0.0
# 3156 The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics Rating: -0.0
# 3157 All Quiet on the Western Front Rating: -0.0
# 3158 Dark Matter Rating: -0.0
# 3159 Atlas Shrugged Rating: -0.0
# 3160 In Five Years Rating: -0.0

In [24]:
#doing masked autoencoder
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, TensorDataset, random_split

# Mask for observed values (1 for observed, 0 for missing)
ratings_torch = torch.tensor(ratings).float()
mask = (ratings_torch != 0).float()
mask_tensor = torch.tensor(mask)


#Define autoencoder
class SparseAutoencoder(nn.Module):
    def __init__(self, num_items, latent_dim):
        super(SparseAutoencoder, self).__init__()
        self.encoder = nn.Linear(num_items, latent_dim)
        self.decoder = nn.Linear(latent_dim, num_items)
        
    def forward(self, x):
        encoded = torch.relu(self.encoder(x))
        decoded = self.decoder(encoded)
        # Scale sigmoid output to [1, 5]
        return 1 + 4 * torch.sigmoid(decoded)
        return decoded

    
#initialize the model
num_users, num_items = ratings_torch.shape
np.save("num_users.npy", np.array(num_users))
np.save("num_items.npy", np.array(num_items))
latent_dim = 100 # Number of latent features

model = SparseAutoencoder(num_items, latent_dim)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Use MSE loss but only consider observed values
def masked_mse_loss(reconstructed, original, mask):
    loss = ((reconstructed - original) ** 2) * mask
    return loss.sum() / mask.sum()

#break up data into train and val
dataset = TensorDataset(ratings_torch, mask_tensor) #keeping the mask
print(len(dataset))
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print("len(train_loader) = ", len(train_loader))
print("len(val_loader) = ", len(val_loader))

#train the model
epochs = 5000
best_loss = 1.0359#10e10
counter = 0
for epoch in range(epochs):
    train_loss = 0.0
    for inputs, mask in train_loader:
        model.train()
        optimizer.zero_grad()

        # Forward pass
        reconstructed = model(inputs)
        loss = masked_mse_loss(reconstructed, inputs, mask)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()

    train_loss /= len(train_loader)
    
    if (epoch + 1) % 10 == 0:
#         print(f"Epoch {epoch + 1}/{epochs}, Loss: {train_loss}")
        #check validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, mask in val_loader:
                outputs = model(inputs)
                loss = masked_mse_loss(outputs, inputs, mask)
                val_loss += loss.item()
                

        val_loss /= len(val_loader)
        

        print(f"Epoch {epoch + 1} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")
        if val_loss < best_loss: #if improve then save
            #save model 
            torch.save(model.state_dict(), "model.pkl")
            best_loss = val_loss
            print("Model saved to model.pkl.")
            counter = 0
        else:
            counter += 10
            
    if counter > 200:
        print("Done training because of no improvement.")
        break
        
            


/tmp/ipykernel_1817/2993602181.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask_tensor = torch.tensor(mask)


767
len(train_loader) =  20
len(val_loader) =  5
Epoch 10 - Train Loss: 0.5757 - Val Loss: 1.2393
Epoch 20 - Train Loss: 0.4498 - Val Loss: 1.1786
Epoch 30 - Train Loss: 0.4586 - Val Loss: 1.1662
Epoch 40 - Train Loss: 0.4307 - Val Loss: 1.1587
Epoch 50 - Train Loss: 0.5087 - Val Loss: 1.1299
Epoch 60 - Train Loss: 0.4764 - Val Loss: 1.1373
Epoch 70 - Train Loss: 0.5345 - Val Loss: 1.1284
Epoch 80 - Train Loss: 0.5568 - Val Loss: 1.1339
Epoch 90 - Train Loss: 0.5284 - Val Loss: 1.1259
Epoch 100 - Train Loss: 0.5788 - Val Loss: 1.1314
Epoch 110 - Train Loss: 0.5134 - Val Loss: 1.1198
Epoch 120 - Train Loss: 0.5332 - Val Loss: 1.1202
Epoch 130 - Train Loss: 0.5493 - Val Loss: 1.1168
Epoch 140 - Train Loss: 0.5719 - Val Loss: 1.1106
Epoch 150 - Train Loss: 0.5697 - Val Loss: 1.0902
Epoch 160 - Train Loss: 0.5690 - Val Loss: 1.0980
Epoch 170 - Train Loss: 0.5759 - Val Loss: 1.0842
Epoch 180 - Train Loss: 0.6145 - Val Loss: 1.0931
Epoch 190 - Train Loss: 0.5849 - Val Loss: 1.0889
Epoch 200 

In [18]:
num_users, num_items

(767, 3239)

In [19]:
print(sum(sum(mask)))
print(mask.shape)
print(mask.shape[0] * mask.shape[1])
print(sum(sum(mask)) / (mask.shape[0] * mask.shape[1]))



tensor(1455.)
torch.Size([26, 3239])
84214
tensor(0.0173)


In [20]:
#Evaulating the model
model.eval()
with torch.no_grad():
    reconstructed = model(ratings_torch)

# Fill missing values in the original matrix
filled_data = ratings_torch.clone()
filled_data[mask_tensor == 0] = reconstructed[mask_tensor == 0]

print("Original Data:\n", ratings_torch)
print("Reconstructed Data:\n", reconstructed)
print("Filled Data:\n", filled_data)


Original Data:
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
Reconstructed Data:
 tensor([[4.2671, 4.9951, 3.3982,  ..., 4.9992, 1.8067, 5.0000],
        [2.7832, 4.2727, 4.7912,  ..., 4.5864, 4.0000, 3.0635],
        [2.7832, 4.2727, 4.7912,  ..., 4.5864, 4.0000, 3.0635],
        ...,
        [2.7832, 4.2727, 4.7912,  ..., 4.5864, 4.0000, 3.0635],
        [3.8883, 4.5733, 4.8944,  ..., 4.9960, 3.7647, 2.7165],
        [2.7832, 4.2727, 4.7912,  ..., 4.5864, 4.0000, 3.0635]])
Filled Data:
 tensor([[4.2671, 4.9951, 3.3982,  ..., 4.9992, 1.8067, 5.0000],
        [2.7832, 4.2727, 4.7912,  ..., 4.5864, 4.0000, 3.0635],
        [2.7832, 4.2727, 4.7912,  ..., 4.5864, 4.0000, 3.0635],
        ...,
        [2.7832, 4.2727, 4.7912,  ..., 4.5864, 4.0000, 3.0635],
        [3.8883, 4.5733, 4.8

In [21]:
print(reconstructed[0].numpy())
print(ratings_torch[0].numpy())

print((reconstructed[0].numpy()-ratings_torch[0].numpy())/ratings_torch[0].numpy())

[4.267067  4.995062  3.3981714 ... 4.9992075 1.8066964 5.       ]
[0. 0. 0. ... 0. 0. 0.]
[inf inf inf ... inf inf inf]


/tmp/ipykernel_1817/2166569457.py:4: RuntimeWarning: divide by zero encountered in divide
  print((reconstructed[0].numpy()-ratings_torch[0].numpy())/ratings_torch[0].numpy())


In [22]:
pred_ratings_list = reconstructed[0].detach().numpy()

#give a list sorted out with books you've already read:
sorted_indices = np.argsort(pred_ratings_list)[::-1]
print("Top books are:")
list_num = 1
for idx in sorted_indices: 
#     print("ratings_matrix[user_id, idx]= ", ratings_matrix[user_id, idx])
    if  (ratings_matrix[user_id, idx] > 0) or(np.isnan(pred_ratings_list[idx])) :
        continue
    print("#", list_num , titles[idx], " - Predicted Rating:", round(pred_ratings_list[idx], 1))
    list_num += 1

Top books are:
# 1 The â€‹Crown of Gilded Bones
        (Blood and Ash, #3)  - Predicted Rating: 5.0
# 2 Empire of Storms
        (Throne of Glass, #5)  - Predicted Rating: 5.0
# 3 Memory Man
        (Amos Decker, #1)  - Predicted Rating: 5.0
# 4 Ruthless Vows
        (Letters of Enchantment, #2)  - Predicted Rating: 5.0
# 5 Circe  - Predicted Rating: 5.0
# 6 Madeline  - Predicted Rating: 5.0
# 7 Stories I Only Tell My Friends  - Predicted Rating: 5.0
# 8 The Notebook
        (The Notebook, #1)  - Predicted Rating: 5.0
# 9 Then She Was Gone  - Predicted Rating: 5.0
# 10 All the Light We Cannot See  - Predicted Rating: 5.0
# 11 Happy Place  - Predicted Rating: 5.0
# 12 Ø§Ù„Ù‚Ø±Ø¢Ù† Ø§Ù„ÙƒØ±ÙŠÙ…  - Predicted Rating: 5.0
# 13 From Blood and Ash
        (Blood and Ash, #1)  - Predicted Rating: 5.0
# 14 Daughter of Fortune  - Predicted Rating: 5.0
# 15 The Tattooist of Auschwitz
        (The Tattooist of Auschwitz, #1)  - Predicted Rating: 5.0
# 16 The Girl Who Kicked the Hornetâ€™s Nest
  

# 148 Powerless
        (The Powerless Trilogy, #1)  - Predicted Rating: 5.0
# 149 The Bean Trees
        (Greer Family, #1)  - Predicted Rating: 5.0
# 150 Crime and Punishment  - Predicted Rating: 5.0
# 151 Little Bee  - Predicted Rating: 5.0
# 152 The Opal Deception
        (Artemis Fowl, #4)  - Predicted Rating: 5.0
# 153 Station Eleven  - Predicted Rating: 5.0
# 154 The Accidental Tourist  - Predicted Rating: 5.0
# 155 Aristotle and Dante Discover the Secrets of the Universe
        (Aristotle and Dante, #1)  - Predicted Rating: 5.0
# 156 Killers of the Flower Moon: The Osage Murders and the Birth of the FBI  - Predicted Rating: 5.0
# 157 The Blue Sword
        (Damar, #1)  - Predicted Rating: 5.0
# 158 Small Great Things  - Predicted Rating: 5.0
# 159 The Drawing of the Three
        (The Dark Tower, #2)  - Predicted Rating: 5.0
# 160 Hopeless
        (Hopeless, #1)  - Predicted Rating: 5.0
# 161 The Mysterious Benedict Society
        (The Mysterious Benedict Society, #1)  - Pred

        (Foundation, #2)  - Predicted Rating: 5.0
# 299 A Game of Thrones
        (A Song of Ice and Fire, #1)  - Predicted Rating: 5.0
# 300 The Scorch Trials
        (The Maze Runner, #2)  - Predicted Rating: 5.0
# 301 The Five People You Meet in Heaven  - Predicted Rating: 5.0
# 302 The Secret Life of Bees  - Predicted Rating: 5.0
# 303 The Time Traveler's Wife  - Predicted Rating: 5.0
# 304 Night Shift  - Predicted Rating: 5.0
# 305 Madame Bovary  - Predicted Rating: 5.0
# 306 The Ten Thousand Doors of January  - Predicted Rating: 5.0
# 307 The Sound and the Fury  - Predicted Rating: 5.0
# 308 Oryx and Crake
        (MaddAddam, #1)  - Predicted Rating: 5.0
# 309 The Secret Hour
        (Midnighters, #1)  - Predicted Rating: 5.0
# 310 Thunderhead
        (Arc of a Scythe, #2)  - Predicted Rating: 5.0
# 311 Villette  - Predicted Rating: 5.0
# 312 Dolores Claiborne  - Predicted Rating: 5.0
# 313 Miss Peregrine's Home for Peculiar Children
        (Miss Peregrine's Peculiar Children, #

# 469 Corduroy  - Predicted Rating: 5.0
# 470 Obsidian Butterfly
        (Anita Blake, Vampire Hunter, #9)  - Predicted Rating: 5.0
# 471 The Outsider  - Predicted Rating: 5.0
# 472 The Wedding People  - Predicted Rating: 5.0
# 473 The Door  - Predicted Rating: 5.0
# 474 Autobiography of a Face  - Predicted Rating: 5.0
# 475 Sophieâ€™s World  - Predicted Rating: 5.0
# 476 The Sixth Man
        (Sean King & Michelle Maxwell, #5)  - Predicted Rating: 5.0
# 477 At Home: A Short History of Private Life  - Predicted Rating: 5.0
# 478 How the Grinch Stole Christmas!  - Predicted Rating: 5.0
# 479 My Cousin Rachel  - Predicted Rating: 5.0
# 480 Vision in White
        (Bride Quartet, #1)  - Predicted Rating: 5.0
# 481 The Good House  - Predicted Rating: 5.0
# 482 The Bullet That Missed
        (Thursday Murder Club, #3)  - Predicted Rating: 5.0
# 483 The Blue Hour  - Predicted Rating: 5.0
# 484 Brown Girl Dreaming  - Predicted Rating: 5.0
# 485 The Night Ship  - Predicted Rating: 5.0
# 486 Hi

# 618 The Nothing Man  - Predicted Rating: 5.0
# 619 When We Left Cuba
        (The Perez Family, #2)  - Predicted Rating: 5.0
# 620 Lone Wolf  - Predicted Rating: 5.0
# 621 The Death of Mrs. Westaway  - Predicted Rating: 5.0
# 622 I Let You Go  - Predicted Rating: 5.0
# 623 Raise High the Roof Beam, Carpenters & Seymour: An Introduction  - Predicted Rating: 5.0
# 624 Fragile Things: Short Fictions and Wonders  - Predicted Rating: 5.0
# 625 The Last Battle
        (Chronicles of Narnia, #7)  - Predicted Rating: 5.0
# 626 Daisy Darker  - Predicted Rating: 5.0
# 627 Exile
        (Forgotten Realms: The Dark Elf Trilogy, #2; Legend of Drizzt, #2)  - Predicted Rating: 5.0
# 628 Being Mortal: Medicine and What Matters in the End  - Predicted Rating: 5.0
# 629 Saint X  - Predicted Rating: 5.0
# 630 South of the Border, West of the Sun  - Predicted Rating: 5.0
# 631 Congo  - Predicted Rating: 5.0
# 632 Too Late  - Predicted Rating: 5.0
# 633 The Running Man  - Predicted Rating: 5.0
# 634 Torm

...

# 2489 Starling House  - Predicted Rating: 1.3
# 2490 Awakened
        (House of Night, #8)  - Predicted Rating: 1.3
# 2491 Valentine  - Predicted Rating: 1.3
# 2492 The Swimmer  - Predicted Rating: 1.3
# 2493 The Last of the Mohicans
        (The Leatherstocking Tales, #2)  - Predicted Rating: 1.3
# 2494 Wuthering Heights  - Predicted Rating: 1.3
# 2495 The Dispossessed: An Ambiguous Utopia  - Predicted Rating: 1.3
# 2496 Four Past Midnight  - Predicted Rating: 1.3
# 2497 When She Woke  - Predicted Rating: 1.3
# 2498 My Side of the Mountain
        (Mountain, #1)  - Predicted Rating: 1.3
# 2499 The Ruins of Gorlan
        (Ranger's Apprentice, #1)  - Predicted Rating: 1.3
# 2500 People of the Book  - Predicted Rating: 1.3
# 2501 For One More Day  - Predicted Rating: 1.3
# 2502 A Clash of Kings
        (A Song of Ice and Fire, #2)  - Predicted Rating: 1.3
# 2503 Drive Your Plow Over the Bones of the Dead  - Predicted Rating: 1.3
# 2504 Lady Tan’s Circle of Women  - Predicted Rating: 1.

# 2623 Let Us Descend  - Predicted Rating: 1.1
# 2624 Cat on a Hot Tin Roof  - Predicted Rating: 1.1
# 2625 The Whispers  - Predicted Rating: 1.1
# 2626 We Are Not Ourselves  - Predicted Rating: 1.1
# 2627 The Chamber  - Predicted Rating: 1.1
# 2628 Down and Out in Paris and London  - Predicted Rating: 1.1
# 2629 Other Birds  - Predicted Rating: 1.1
# 2630 How the King of Elfhame Learned to Hate Stories
        (The Folk of the Air, #3.5)  - Predicted Rating: 1.1
# 2631 Trust  - Predicted Rating: 1.1
# 2632 The Thousand Autumns of Jacob de Zoet  - Predicted Rating: 1.1
# 2633 The Halfling's Gem
        (Forgotten Realms: Icewind Dale, #3; Legend of Drizzt, #6)  - Predicted Rating: 1.1
# 2634 Redhead by the Side of the Road  - Predicted Rating: 1.1
# 2635 The Dark Forest
        (Remembrance of Earthâ€™s Past, #2)  - Predicted Rating: 1.1
# 2636 The Postcard Killers  - Predicted Rating: 1.1
# 2637 Saturday  - Predicted Rating: 1.1
# 2638 Razorblade Tears  - Predicted Rating: 1.1
# 2639 

# 2791 Darkfever
        (Fever, #1)  - Predicted Rating: 1.0
# 2792 Three Wishes  - Predicted Rating: 1.0
# 2793 Lost Roses
        (Lilac Girls, #2)  - Predicted Rating: 1.0
# 2794 Anne Frank Remembered: The Story of the Woman Who Helped to Hide the Frank Family  - Predicted Rating: 1.0
# 2795 The Rescue  - Predicted Rating: 1.0
# 2796 My Ãntonia  - Predicted Rating: 1.0
# 2797 Eye of the Needle  - Predicted Rating: 1.0
# 2798 Goodnight Beautiful  - Predicted Rating: 1.0
# 2799 All My Puny Sorrows  - Predicted Rating: 1.0
# 2800 The End of the Affair  - Predicted Rating: 1.0
# 2801 Before the Fall  - Predicted Rating: 1.0
# 2802 Aesopâ€™s Fables  - Predicted Rating: 1.0
# 2803 The Hike  - Predicted Rating: 1.0
# 2804 Dear Wife  - Predicted Rating: 1.0
# 2805 Walking Disaster
        (Beautiful, #2)  - Predicted Rating: 1.0
# 2806 Counterfeit  - Predicted Rating: 1.0
# 2807 Play It As It Lays  - Predicted Rating: 1.0
# 2808 The Sea of Monsters
        (Percy Jackson and the Olympians

# 2958 Ethan Frome  - Predicted Rating: 1.0
# 2959 City of Fallen Angels
        (The Mortal Instruments, #4)  - Predicted Rating: 1.0
# 2960 Jemima J  - Predicted Rating: 1.0
# 2961 The Celestine Prophecy
        (Celestine Prophecy, #1)  - Predicted Rating: 1.0
# 2962 Saving Fish from Drowning  - Predicted Rating: 1.0
# 2963 Betrayed
        (House of Night, #2)  - Predicted Rating: 1.0
# 2964 Catch and Kill: Lies, Spies, and a Conspiracy to Protect Predators  - Predicted Rating: 1.0
# 2965 Sense and Sensibility  - Predicted Rating: 1.0
# 2966 City of Ashes
        (The Mortal Instruments, #2)  - Predicted Rating: 1.0
# 2967 The Year of Living Biblically: One Man's Humble Quest to Follow the Bible as Literally as Possible  - Predicted Rating: 1.0
# 2968 The Invisible Man  - Predicted Rating: 1.0
# 2969 The Housemaid's Secret
        (The Housemaid, #2)  - Predicted Rating: 1.0
# 2970 The Quickie  - Predicted Rating: 1.0
# 2971 Test of the Twins
        (Dragonlance: Legends, #3)  - P

# 3140 Like Water for Chocolate  - Predicted Rating: 1.0
# 3141 Assassin's Apprentice
        (Farseer Trilogy, #1)  - Predicted Rating: 1.0
# 3142 The Kite Runner  - Predicted Rating: 1.0
# 3143 The Alchemist  - Predicted Rating: 1.0
# 3144 The Woman in the Window  - Predicted Rating: 1.0
# 3145 Specials
        (Uglies, #3)  - Predicted Rating: 1.0
# 3146 The da Vinci Code
        (Robert Langdon, #2)  - Predicted Rating: 1.0
# 3147 Heart of Darkness  - Predicted Rating: 1.0
# 3148 The Adventures of Huckleberry Finn  - Predicted Rating: 1.0
# 3149 In Cold Blood  - Predicted Rating: 1.0
# 3150 The Shack  - Predicted Rating: 1.0
# 3151 Lolita  - Predicted Rating: 1.0
# 3152 The Well of Ascension
        (Mistborn, #2)  - Predicted Rating: 1.0
# 3153 Lord of Chaos
        (The Wheel of Time, #6)  - Predicted Rating: 1.0
# 3154 Me Talk Pretty One Day  - Predicted Rating: 1.0
# 3155 Yes Please  - Predicted Rating: 1.0
# 3156 The Red Tent  - Predicted Rating: 1.0
# 3157 The Little House Co

In [23]:
dfghj
import torch
from sklearn.model_selection import KFold

# Mask for observed values (1 for observed, 0 for missing)
ratings_torch = torch.tensor(ratings).float()
mask = (ratings_torch != 0).float()
print(mask)


#Define autoencoder
class SparseAutoencoder(nn.Module):
    def __init__(self, num_items, latent_dim):
        super(SparseAutoencoder, self).__init__()
        self.encoder = nn.Linear(num_items, latent_dim)
        self.decoder = nn.Linear(latent_dim, num_items)
        
    def forward(self, x):
        encoded = torch.relu(self.encoder(x))
        decoded = self.decoder(encoded)
        # Scale sigmoid output to [1, 5]
        return 1 + 4 * torch.sigmoid(decoded)
        return decoded

    
#initialize the model
num_users, num_items = ratings_torch.shape
np.save("num_users.npy", np.array(num_users))
np.save("num_items.npy", np.array(num_items))

for latent_dim in [2, 5, 10, 20, 40, 50, 75, 100]:
    print("latent_dim = ", latent_dim)
# latent_dim = 20  # Number of latent features

    model = SparseAutoencoder(num_items, latent_dim)
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    # Define your model, loss function, and optimizer
    # Assuming model, ratings_torch, mask, and optimizer are already defined

    epochs = 1000
    k_folds = 5  # Number of folds for cross-validation
    kf = KFold(n_splits=k_folds, shuffle=True)

    # Store the losses for each fold
    fold_losses = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(ratings_torch)):
#         print(f"\nFold {fold + 1}/{k_folds}")

        # Split the data into training and validation sets
        train_ratings = ratings_torch[train_idx]
        val_ratings = ratings_torch[val_idx]
        train_mask = mask[train_idx]
        val_mask = mask[val_idx]

        # Initialize a new model for each fold
        model = SparseAutoencoder(num_items, latent_dim)
    #     optimizer = optim.Adam(model.parameters(), lr=0.01)

        # Re-initialize optimizer for each fold
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

        # Train the model
        for epoch in range(epochs):
            model.train()
            optimizer.zero_grad()

            # Forward pass for training
            reconstructed = model(train_ratings)
            loss = masked_mse_loss(reconstructed, train_ratings, train_mask)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

#             if (epoch + 1) % 1000 == 0:
#                 print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}")

        # Evaluate the model on the validation set
        model.eval()
        with torch.no_grad():
            reconstructed_val = model(val_ratings)
            val_loss = masked_mse_loss(reconstructed_val, val_ratings, val_mask)

        print(f"Validation Loss for Fold {fold + 1}: {val_loss.item():.4f}")

        # Store the validation loss for this fold
        fold_losses.append(val_loss.item())

    # Print the average validation loss after all folds
    print(f"\nAverage Validation Loss across all folds: {sum(fold_losses)/k_folds:.4f}")


NameError: name 'dfghj' is not defined

In [ ]:
fgh

In [ ]:
ratings_matrix[user_id]

In [ ]:
# Convert the filled ratings data into a numpy array
ratings_matrix = ratings_df.values

# Initialize KNN (using user-based KNN)
import math
knn = NearestNeighbors(n_neighbors=math.ceil(num_users/10), metric='cosine')  # Using cosine similarity
knn.fit(ratings_matrix)

# Example: Predict rating for User 0 on Item 2
user_id = 0  # Index of user in the matrix
item_id = 9  # Index of item to predict rating for

# Get the nearest neighbors for user 0 (excluding the user itself)
distances, indices = knn.kneighbors([ratings_matrix[user_id]])

pred_ratings_list = np.array([])
rankings_list = np.array([])
for item_id in range(num_titles):
    # Get the ratings for the neighbors on item 2
    neighbor_ratings = np.array([ratings_matrix[i, item_id] for i in indices[0] if not np.isnan(ratings_matrix[i, item_id])])
    print(ratings_matrix[:, item_id])
    print(neighbor_ratings)
    ghjk
    
    predicted_rating = np.mean(neighbor_ratings[np.nonzero(neighbor_ratings)])
    rankings = np.sum(neighbor_ratings[np.nonzero(neighbor_ratings)])

#     print(predicted_rating)

    pred_ratings_list = np.append(pred_ratings_list, predicted_rating)
    rankings_list = np.append(rankings_list, rankings)
#     print(f"Predicted rating for User {user_id} on Item {item_id}: {predicted_rating}")
    
best_book_rating = np.max(pred_ratings_list)
best_book_idx = np.argmax(pred_ratings_list)

sorted_indices = np.argsort(pred_ratings_list)[::-1]
print("Top books are:")
for i, idx in enumerate(sorted_indices): 
    print("#", (i+1) , titles[idx], "Rating:", round(pred_ratings_list[idx], 1))